<a href="https://colab.research.google.com/github/AlanMarket/FirstRepository/blob/master/NueralNet_vs_LinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings

%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
data = pd.read_csv("https://s3.amazonaws.com/vargo.aprd6342/data/trainingdata.csv", encoding="latin-1")

In [0]:
data.head()

,Brand Mention?,userpost,match type,brand
0,1,RT @Dodge: Show us your stripes. (Photo credit...,vague,dodge
1,1,I took out the E-Rep! Are you up to the challe...,vague,dodge
2,1,I'm earning #mPLUSRewards in Dodge Dot. https:...,vague,dodge
3,0,Clouds... What I wouldn't give for Instagram t...,vague,dodge
4,1,RT @Dodge: Best lights on the block. https://t...,vague,dodge


In [0]:
data.columns

Index(['Brand Mention?', 'userpost', 'match type', 'brand'], dtype='object')

In [0]:
data['match type'].head()

0    vague
1    vague
2    vague
3    vague
4    vague
Name: match type, dtype: object

In [0]:
one_hot = pd.get_dummies(data, columns=['match type', 'brand'])

In [0]:
one_hot.head()

,Brand Mention?,userpost,match type_unique,match type_vague,brand_1800,brand_5-hour,brand_aarp,brand_acura,brand_advil,brand_airborne,brand_aleve,brand_allegra,brand_allstate,brand_android,brand_applebees,brand_arbys,brand_atkins,brand_audi,brand_bacardi,brand_band-aid,brand_beautyrest,brand_bebe,brand_belvita,brand_bing,brand_bmw,brand_bosch,brand_botox,brand_bounty,brand_bridgestone,brand_brita,brand_brookside,brand_budweiser,brand_buick,brand_cadillac,brand_camel,brand_cartier,brand_celine,brand_centurylink,brand_chanel,brand_cheerios,...,brand_subway,brand_swanson,brand_swiffer,brand_tempur-pedic,brand_tena,brand_tide,brand_tiffany,brand_tostitos,brand_toyota,brand_transitions,brand_travelers,brand_trident,brand_tripadvisor,brand_triscuit,brand_trix,brand_tropicana,brand_trumoo,brand_truvia,brand_twinings,brand_twix,brand_twizzlers,brand_tylenol,brand_tyson,brand_usps,brand_v8,brand_valentino,brand_verizon,brand_versace,brand_viagra,brand_visa,brand_viva,brand_volkswagen,brand_volvo,brand_wendys,brand_wrangler,brand_xfinity,brand_yoplait,brand_zicam,brand_ziploc,brand_zzzquil
0,1,RT @Dodge: Show us your stripes. (Photo credit...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,I took out the E-Rep! Are you up to the challe...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,I'm earning #mPLUSRewards in Dodge Dot. https:...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,Clouds... What I wouldn't give for Instagram t...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,RT @Dodge: Best lights on the block. https://t...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
one_hot['match type_vague']

0       1
1       1
2       1
3       1
4       1
       ..
3290    0
3291    0
3292    0
3293    0
3294    0
Name: match type_vague, Length: 3295, dtype: uint8

In [0]:
tweetTokens = Tokenizer(num_words= 1000)
text_data = one_hot[['userpost']].values.reshape(1,-1)[0]

In [0]:
text_data

array(['RT @Dodge: Show us your stripes. (Photo credit: Nichole W.) #ThatsMyDodge https://t.co/1fDyrT47Kp',
       'I took out the E-Rep! Are you up to the challenge? #Defiance http://t.co/9NLFUUnJTB @DefianceWorld @Dodge',
       "I'm earning #mPLUSRewards in Dodge Dot. https://t.co/LITjWyJSDU",
       ...,
       'Me and the BIL #beastmode #GOHAWKS #Seahawks @ CenturyLink Field http://t.co/ClBoWHVu6c',
       '#GOHAWKS @ CenturyLink Field http://t.co/hpdWr8lizJ',
       'OK TODAY LIKE MARCH 23RD 320PM  I THINK  TO ALL----------------  WELL INTERNET BEEN MOSTLY OFF TODAY DUE TO CENTURYLINK  NO BIGGIE ROMEO AND ME JUST SAT OUTSIDE LONGTIMETIME ENJOYINNG LIFE BUT GOT TO CHILLY WE NOW BACK INSIDE ROMEO AND ME BEEN STOCKING UP FOR ZOMBIE APOCALYPE OR ANY APOCALYPSE  I THINK ROMEO GOT MORE THEN I DO BUT HEY IF IT CAME DOWN TO IT ILL EAT HIS FOOD HA HA L;OL  NO WE ARE DOING GREAT  THOUGH I WONDER WHAT THE RULES ARE FR SENDING BOX TO MADAGASCAR W SOME FUR STICKING OUTTTA IT HA HA LOL  SO YUP

In [0]:
tweetTokens.fit_on_texts(text_data)

In [0]:
word_matrix = tweetTokens.texts_to_matrix(text_data)

otherPredictors = one_hot.drop('Brand Mention?', axis = 1)

In [0]:
otherPredictors.head()

,userpost,match type_unique,match type_vague,brand_1800,brand_5-hour,brand_aarp,brand_acura,brand_advil,brand_airborne,brand_aleve,brand_allegra,brand_allstate,brand_android,brand_applebees,brand_arbys,brand_atkins,brand_audi,brand_bacardi,brand_band-aid,brand_beautyrest,brand_bebe,brand_belvita,brand_bing,brand_bmw,brand_bosch,brand_botox,brand_bounty,brand_bridgestone,brand_brita,brand_brookside,brand_budweiser,brand_buick,brand_cadillac,brand_camel,brand_cartier,brand_celine,brand_centurylink,brand_chanel,brand_cheerios,brand_cheetos,...,brand_subway,brand_swanson,brand_swiffer,brand_tempur-pedic,brand_tena,brand_tide,brand_tiffany,brand_tostitos,brand_toyota,brand_transitions,brand_travelers,brand_trident,brand_tripadvisor,brand_triscuit,brand_trix,brand_tropicana,brand_trumoo,brand_truvia,brand_twinings,brand_twix,brand_twizzlers,brand_tylenol,brand_tyson,brand_usps,brand_v8,brand_valentino,brand_verizon,brand_versace,brand_viagra,brand_visa,brand_viva,brand_volkswagen,brand_volvo,brand_wendys,brand_wrangler,brand_xfinity,brand_yoplait,brand_zicam,brand_ziploc,brand_zzzquil
0,RT @Dodge: Show us your stripes. (Photo credit...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,I took out the E-Rep! Are you up to the challe...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,I'm earning #mPLUSRewards in Dodge Dot. https:...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Clouds... What I wouldn't give for Instagram t...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,RT @Dodge: Best lights on the block. https://t...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
otherPredictors = otherPredictors.drop('userpost', axis = 1)

otherPredictorsMatrix = otherPredictors.values

In [0]:
otherPredictorsMatrix

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [0]:
#word_matrix = tweetTokens.texts_to_matrix(text_data)

#otherPredictors = one_hot.drop('Brand Mention?', axis = 1)
#otherPredictors = otherPredictors.drop('userpost', axis = 1)

#otherPredictorsMatrix = otherPredictors.values
_X = np.concatenate((word_matrix, otherPredictorsMatrix),axis = 1)

_X = np.asarray(_X)



_y = data['Brand Mention?']
_y = np.asarray(_y)


In [0]:
_X

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
model = Sequential()

dimensions = _X.shape[1]

#adding the first layer, "relu" refers to how a node is activated
model.add(Dense(64, input_dim=dimensions, activation="relu"))

# additional layer
model.add(Dense(1, activation="sigmoid"))


model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                81792     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 81,857
Trainable params: 81,857
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='mse', optimizer='rmsprop',metrics=['mse', 'accuracy']) # Metrics only correspond to the test set

model.fit(_X,_y,batch_size=_X.shape[0],epochs=20, shuffle=True, validation_split=0.3)


Train on 2306 samples, validate on 989 samples
Epoch 1/20
2306/2306 [==============================] - 0s 63us/sample - loss: 0.2533 - mean_squared_error: 0.2533 - acc: 0.4679 - val_loss: 0.2224 - val_mean_squared_error: 0.2224 - val_acc: 0.7513
Epoch 2/20
2306/2306 [==============================] - 0s 14us/sample - loss: 0.2237 - mean_squared_error: 0.2237 - acc: 0.7229 - val_loss: 0.2076 - val_mean_squared_error: 0.2076 - val_acc: 0.8018
Epoch 3/20
2306/2306 [==============================] - 0s 14us/sample - loss: 0.2069 - mean_squared_error: 0.2069 - acc: 0.7567 - val_loss: 0.1972 - val_mean_squared_error: 0.1972 - val_acc: 0.8170
Epoch 4/20
2306/2306 [==============================] - 0s 15us/sample - loss: 0.1944 - mean_squared_error: 0.1944 - acc: 0.7654 - val_loss: 0.1896 - val_mean_squared_error: 0.1896 - val_acc: 0.8241
Epoch 5/20
2306/2306 [==============================] - 0s 15us/sample - loss: 0.1841 - mean_squared_error: 0.1841 - acc: 0.7823 - val_loss: 0.1836 - val_mea

In [0]:
warnings